PDF Q&A section extractor

In [1]:
# Pathways
import os
import sys
from pathlib import Path

# Automatically set the project path to the current working directory
project_path = Path.cwd()
os.chdir(project_path)
sys.path.insert(0, str(project_path))

print(f"Current directory: {os.getcwd()}")

Current directory: c:\Users\joshu\OneDrive\Documents\1 Work\Bank of England NLP\Bank-of-England-NLP-on-Earnings-Calls


In [ ]:
import pdfplumber
import os
import csv
from PyPDF2 import PdfWriter, PdfReader

# List of uploaded PDF files
pdf_files = {
    "Q1_Transcript-Analyst-Call-25-April-2024.pdf": "/mnt/data/Q1_Transcript-Analyst-Call-25-April-2024.pdf",
    "Q2_Transcript-Analyst-Call-24-July-2024-For-Upload.pdf": "/mnt/data/Q2_Transcript-Analyst-Call-24-July-2024-For-Upload.pdf",
    "Q3-2024-Analyst-Call-Transcript-For-Upload.pdf": "/mnt/data/Q3-2024-Analyst-Call-Transcript-For-Upload.pdf",
    "jpm-1q24-earnings-call-transcript.pdf": "/mnt/data/jpm-1q24-earnings-call-transcript.pdf",
    "jpm-3q23-earnings-call-transcript.pdf": "/mnt/data/jpm-3q23-earnings-call-transcript.pdf",
    "jpm-3q24-earnings-call-transcript.pdf": "/mnt/data/jpm-3q24-earnings-call-transcript.pdf",
}

# Directory to save extracted Q&A PDFs
output_dir = "/mnt/data/extracted_qna/"
os.makedirs(output_dir, exist_ok=True)

# CSV file to store metadata
csv_file_path = "/mnt/data/qna_sections_metadata.csv"
csv_metadata = []

# Variations of Q&A section title
qna_variations = [
    "Questions and Answers", "QUESTION AND ANSWER SECTION", "Q&A Section", "Questions & Answers"
]

# Function to find the Q&A section start page
def find_qna_section(pdf_path, keywords):
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text and any(keyword in text for keyword in keywords):
                return i  # Return the page number where Q&A starts
    return None

# Function to extract and save Q&A section along with the cover page
def extract_qna_with_cover(pdf_path, qna_page, output_filename):
    with open(pdf_path, "rb") as pdf_file:
        reader = PdfReader(pdf_file)
        writer = PdfWriter()
        
        # Ensure cover page is included
        pages_to_extract = [0, qna_page] if qna_page < len(reader.pages) else [0]

        for page_num in pages_to_extract:
            writer.add_page(reader.pages[page_num])

        new_pdf_path = os.path.join(output_dir, output_filename)
        with open(new_pdf_path, "wb") as output_pdf:
            writer.write(output_pdf)

    return new_pdf_path

# Process each document and save the extracted Q&A section
for original_filename, pdf_path in pdf_files.items():
    qna_page = find_qna_section(pdf_path, qna_variations)
    if qna_page is not None:
        qna_filename = f"QnA_{original_filename}"
        qna_file_path = extract_qna_with_cover(pdf_path, qna_page, qna_filename)

        # Store metadata for CSV
        csv_metadata.append([original_filename, qna_page + 1, qna_filename])

# Write metadata to CSV
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Original File Name", "Q&A Section Page", "Extracted Q&A File Name"])
    writer.writerows(csv_metadata)

# Return the CSV file path for reference
csv_file_path
